首先我们来看看图片中的物体的一些性质：


本节大部分图片选取自Yjango大佬的专栏超智能体(https://zhuanlan.zhihu.com/p/27642620)

1. 画面不变性
我们知道一个物体不管在画面左侧还是右侧，都会被识别为同一物体，这一特点就是不变性，如下图所示。

![](pic\1.png)

而我们之前的全连接神经网络，并没有很明显的体现出对这些特性的支持。

“方便起见，我们用depth只有1的灰度图来举例。 想要完成的任务是：在宽长为4x4的图片中识别是否有下图所示的“横折”。 图中，黄色圆点表示值为0的像素，深色圆点表示值为1的像素。 我们知道不管这个横折在图片中的什么位置，都会被认为是相同的横折。”

![](pic\2.png)

若训练只有全连接层构成的神经网络来完成该任务，那么表达图像的三维张量将会被摊平成一个向量，作为网络的输入，即(width, height, depth)为(4, 4, 1)的图片会被展成维度为16的向量作为网络的输入层。再经过几层不同节点个数的隐藏层，最终输出两个节点，分别表示“有横折的概率”和“没有横折的概率”，如下图所示。
![](pic\3.png)

下面我们用数字（16进制）对图片中的每一个像素点（pixel）进行编号。 当使用右侧那种物体位于中间的训练数据来训练网络时，网络就只会对编号为5,6,9,a的节点的权重进行调节。 若让该网络识别位于右下角的“横折”时，则无法识别。
![](pic\4.png)



我们更希望能有一种新的网络，能满足全部或部分上面提到的特点，因此，卷积神经网络在上世纪80~90年代被提出。(有说法是卷积神经网络的提出是参考了猫的视觉神经结构。)

这个网络层的名称叫做卷积层，就是因为这一层主要做的就是卷积。这里，我们不去纠结卷积是什么，而是着重于卷积的作用是什么，它怎么去计算。

深度学习中的卷积分为1D, 2D, 3D，分别表示在1维数据、2维数据、3维数据上做卷积。我们平时处理的图片一般是看成2维的，虽然图片还有一个通道维度(比如RGB)，但是因为每一个像素我们看成一个整体，因此通道维度我们就不考虑在卷积的方向上。

深度学习中，我们一般认为卷积的作用是做特征提取，或者说卷积神经网络有着类似编码器一样的作用，至于为什么这么说，我们还要从卷积层的运算上看起。

全连接的缺点就是，即使是相同的东西，我们不同的位置对应的权重却不同，那我们希望的就是有没有什么办法，让不同位置也能有相同的权重。卷积层的提出就解决了这个问题。

## 权重在空间上的共享

### 局部连接

全连接层就是因为上一层和下一层之间的每个神经元都有连接，因此出现了这种权重不在空间上共享的情况(因为每一个连接都是一个新的权重，这样的话，任意两个神经元之间的权重都是独一无二，互相独立的。)

而我们提到的权重共享，就是要把相同的权重(对应着相同的边)，连接在前后两层的神经元上，这样的话，就不能像全连接那样了，应该改为局部连接：
![](pic\5.png)

如图，这个带有连接强弱的红色方框就叫做 滤波器(filter) 或 (卷积)核(kernel) 或 特征提取器(feature detector)。 而核的范围叫做核尺寸(filter size)，这里所展示的是2x2的filter size。之前说了，卷积一般是用来提取特征的，那么用来提取特征工具的就是卷积和核了。

卷积核可以用矩阵进行表示: $$\begin{bmatrix}
w_1 & w_2\\ 
w_3 & w_4
\end{bmatrix}$$

需要注意，我们图片上的像素存在着空间上的联系，比如一个像素，他的上下左右都和他是相关的，而如果我们使用全连接的话(会把图片展平)，即输入输出都是1维张量，这样的话，我们会丢失图片的空间信息(无法知道那些像素之间是相邻的)。因此卷积的优势显而易见。

第二层的节点0的数值就是局部区域的线性组合，即被圈中节点的数值乘以对应的权重后相加。 $x$表示输入值，$y$表示输出值，用图中标注数字表示角标，则下面列出了两种计算编号为0的输出值$y_0$的表达式。

$$y_0 = \begin{bmatrix}
w_1 & w_2\\ 
w_3 & w_4
\end{bmatrix} \cdot  \begin{bmatrix}
x_0 & x_1\\ 
x_4 & x_5
\end{bmatrix} + b$$

需要注意的是这里的$\cdot$运算不是矩阵乘法，而是一种类似点积的运算，我们想象一种矩阵的点积，两个做运算的矩阵长宽相等，则我们对应元素相乘后求和作为最终结果。

可以看到，卷积核**提取特征**的的过程就是局部区域上的线性组合(对比于全连接的全局线性组合)，之后也加上了一个偏置b。

如果图片中的某一个区域和卷积核的内容相似的话，则我们运算出来的结果$y$就会比较高(点积的结果可以代表相似度，运算的两个向量共线且方向相同时，结果最大；正交时为0；方向相反时最小)，即图片的特征和卷积核有着较好的匹配，因此对应的神经元会被较好的激活，这就是卷积和可以提取特征的解释。

### 空间共享
上面说了卷积核提取特征的过程，现在就需要把这个过程运用在图片上。根据之前提到的图片的平移不变性，我们知道一个东西的特征可能出现在图片的任意位置上，为了找到这个特征，我们的卷积核也需要在图像上进行移动匹配，这个过程就像就像我们看书时，视线从左到右从上到下进行扫描一样。同样，我们看书有的时候看的比较快，有时则比较满。这在移动的过程中对应了步长strides这一信息，也就是卷积和每一次平移的距离。这里举的图例都是步长为1的情况。
![](pic\6.gif)

因为需要保留图片的空间信息，我们最后的输出也会是类似矩阵的形式而不是向量：
![](pic\6.png)

最终，我们卷积神经网络的计算过程可以总结为下面这张图：
![](pic\7.gif)

# 如何针对多通道图片？如何提取更多的特征？如何体现深度？

上面举的例子，都是单通道图片，又叫灰度图(比如我们MNIST就是灰度图)

而我们现实中，更多会遇到彩色图，彩色图的实现呢，就是把每个像素存储为一个维度为3的向量，这个3维的向量分别记录了这个像素的红色值(red)、绿色值(green)、蓝色值(bule)，因此我们通常称为rgb值。

这样的话，我们每张rgb图片就可以看成一个3维的张量，比如一张$1920 \times 1080$的图片，可以表现成一个$1920\times1080\times3$的张量

我们卷积核提取特征的过程就是在图片上的某一个矩形区域内做线性计算，也就是同时考虑这个区域中的所有像素，当我们图片的深度(通道数)为1时，我们卷积和的深度也为1，那么同理，我们图片变成了RGB图片(通道数为3)，我们的卷积和的深度也就变成了3。比如，当我们的卷积核尺寸为$2 \times 2$时，卷积核的shape其实就是$2 \times 2 \times 3$
![](pic\8.gif)

其次是另一个问题，如何提取更多的特征？

我们知道，因为权重共享的缘故，一个卷积核只能代表一种特征，而我们一张图片，不可能只有一种特征吧？(比如人脸，有一双眼睛，还有鼻子嘴巴耳朵等等，这些都算特征)

那如果我们一张图片希望提取更多的特征怎么办？这个其实很简单，那就加卷积核呗，1个卷积核，我们的输出的特征图深度(层数)是1，那么当有n个卷积核的时候，我们输出的特征图，深度就是n，特征图的每一层都是一个卷积核在图像上做卷积得到的结果。

![](pic\8.png)

第三个问题，卷积神经网络如何体现深度的概念？

在全连接中，我们看到，增加网络的层数，我们网络的表现会变得更好。那么我们卷积层因该也是类似，这里我们从卷积层提取特征的角度解释一下为什么。

先来说一个类似“特征分解”的概念，比如我们的汉字，虽然每个汉字都是一个整体，但是大部分汉字都可以进行细分，比如“卷积的积”，左边一个禾，右边一个只，禾呢，有由不同的笔画构成。我们小学学习写字的过程，肯定都是先学笔画，后学偏旁，最后学更复杂的字。

我们卷积神经网络学习的过程也是如此，我们图像中的低级特征(边缘特征)，就象征着笔画，这些特征一般都很简单，但却是构成更复杂的形状的基础。我们图像更高级的特征(就像汉字的偏旁一样)，都是由简单的更低级的特征构成。

我们卷积的过程，不仅提取了特征，还保留了各特征之间的空间特征。我们每增加一层卷积层，都相当于是对下一层更低级的特征(这里的高级低级是相对的)进行匹配提取组合，形成更高级的，更抽象的特征的过程。以一个图来举例：

![](pic\1.jpg)

### 加入非线性
还记得全连接时提到的么？多层的线性全连接其实只等价于1层的全连接，我们卷积这里也是类似的，为了增强我们卷积的能力，我们也会对卷积的输出做非线性变换，常用的激活函数是ReLu。

# 看keras支持的卷积

keras提供了keras.layers.Conv(X)D来为模型加入卷积层，X可取1，2，3，分别代表卷积的方向数。

一般做图片上的卷积，使用的是Conv2D。

Conv2D的参数很多，我们挑一些主要的进行讲解。

 - filters: Integer, the dimensionality of the output space. 这个就是我们卷积核的个数
 - kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions.
 - strides: An integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Can be a single integer to specify the same value for all spatial dimensions. Specifying any stride value != 1 is incompatible with specifying any dilation_rate value != 1.
 - padding: one of "valid" or "same". 
 - data_format: A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. channels_last corresponds to inputs with shape (batch, height, width, channels) while channels_first corresponds to inputs with shape (batch, channels, height, width). It defaults to the image_data_format value found in your Keras config file at ~/.keras/keras.json. If you never set it, then it will be "channels_last". 这个一般不太需要管
 - activation: Activation function to use. If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).
 - use_bias: Boolean, whether the layer uses a bias vector. 
 
其中出现了两个我们之前没提到的东西：padding和data_format。padding是填充的意思，我们前面举的所有例子都是以valid的方式进行计算。

那么为什么需要padding呢？这是因为卷积操作存在两个问题：
- 1、图像越来越小；
- 2、图像边界信息丢失，即有些图像角落和边界的信息发挥作用较少。

![](pic\7.gif)

因此需要padding

valid这种方式其实并没有填充，也就是我们之前一直看到的，因此这里讲一下same方式。
![](pic\2.jpg)

如果卷积的步长为1时，我们使用same的方式，得到的输出的尺寸是和输入相同的，因此叫做same。不过让步长大于1时，比如为2，我们的输出尺寸会等于输入尺寸//2(除后向上取整)。

至于data_format，这个就是指定我们输入图片的格式。我们大部分时候的图片都是(width, height, channels)，但也有些时候，我们的数据会以(channels, width, height)的方式保存，这里就是去指定我们channel所在的维度是"channels_last"还是"channels_first"。

ok，我们来看一下，我们输入图片的尺寸一般是这样的，[batch_size, old_w, old_h, old_c]，若卷积层中卷积核的个数为new_c个，卷积和的尺寸为size，步长为stride，填充为valid的话，则输出的尺寸应该是[batch_size, new_w, new_h, new_c]，其中,$W_{new}=(W_{old}-size)/stride + 1)$, $H_{new}=(H_{old}-size)/stride + 1)$

若padding是same，则$W_{new}=W_{old}//stride$, $H_{new}=H_{old}//stride$

In [2]:
import tensorflow.keras as keras
import tensorflow as tf

a = tf.random.normal(shape=[1, 7, 7, 3]) # 假设这是一张7*7的3通道图片

conv1 = keras.layers.Conv2D(5, 3, 2, 'valid')(a)
conv2 = keras.layers.Conv2D(5, 3, 2, 'same')(a)

print(conv1.shape)
print(conv2.shape)

(1, 3, 3, 5)
(1, 4, 4, 5)


使用在MNIST上的话，我们可以先用卷积层提取数字的特征，当提取到较深层的特征后，我们就可以使用全连接进行分类(1是我们输出的是一个表示概率分布的向量，输出向量一般使用全连接，2是虽然全连接不善于处理图片，但善于分类，一般做图像分类的模型，最后都会接上全连接)

In [7]:
import input_data

dataset = input_data.read_data_sets('mnist_data/', one_hot=True)

train_dataset = dataset.train
test_dataset = dataset.test

input_ = keras.Input(shape=(784, ))
input_reshape = tf.reshape(input_, (-1, 28, 28, 1)) # 一定要包括深度维

conv1 = keras.layers.Conv2D(16, 3, 2, activation='relu')(input_reshape)
conv2 = keras.layers.Conv2D(32, 3, 2, activation='relu')(conv1)
conv3 = keras.layers.Conv2D(64, 3, 2, activation='relu')(conv2)

flatten = keras.layers.Flatten()(conv3) # 因为全连接的输入是向量，所以要先把特征展开成向量

fc1 = keras.layers.Dense(128, activation='relu')(flatten)
result = keras.layers.Dense(10, activation='softmax')(fc1)

model = keras.Model(input_, result)
model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

model.fit(train_dataset.images, train_dataset.labels, batch_size=32, epochs=20)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Train on 55000 samples
Epoch 1/20
55000/55000 [==============================] - 8s 141us/sample - loss: 0.2130 - accuracy: 0.9354
Epoch 2/20
55000/55000 [==============================] - 7s 128us/sample - loss: 0.0743 - accuracy: 0.9771
Epoch 3/20
55000/55000 [==============================] - 7s 126us/sample - loss: 0.0528 - accuracy: 0.9838
Epoch 4/20
55000/55000 [==============================] - 7s 126us/sample - loss: 0.0414 - accuracy: 0.9874
Epoch 5/20
55000/55000 [==============================] - 7s 127us/sample - loss: 0.0352 - accuracy: 0.9895
Epoch 6/20
55000/55000 [==============================] - 7s 126us/sample - loss: 0.0303 - accuracy: 0.9909
Epoch 7/20
55000/55000 [==============================] - 7s 129us/sample - loss: 0.0265 - accuracy: 0.9924
Epoch 8/20
55000/55000 [===

In [8]:
model.evaluate(test_dataset.images, test_dataset.labels)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.0859517462794723, 0.9824]

只玩MNIST没意思？我们之后换一个数据集：手写汉字(滑稽)

数据集下载地址：http://www.nlpr.ia.ac.cn/databases/handwriting/Home.html

要下载的文件：

- HWDB1.1trn_gnt（1873MB）
- HWDB1.1tst_gnt（471MB）